# Attachment Protocol Example -Bob


In [28]:
%autoawait
import time
import asyncio
import base64

IPython autoawait is `on`, and set to use `asyncio`


#### Intialise Controller

In [29]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8060
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

#### Check if active connection exists

In [30]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


Results :  [{'state': 'active', 'created_at': '2020-09-18 11:03:22.682793Z', 'connection_id': '97ef2879-7643-4e4f-9141-7d9edfa49de5', 'routing_state': 'none', 'their_did': 'MmdMDrUhYfHnF2yxH4Bfyb', 'invitation_mode': 'once', 'their_label': 'Alice', 'my_did': '5j6W5xo2aPPzXKKinSQc5E', 'initiator': 'self', 'updated_at': '2020-09-18 11:03:47.287292Z', 'invitation_key': 'EJv5a7NDdMZJ196SdmTpZtEt7VnmiFUvkpEVfdc4wnLu', 'accept': 'manual'}]
Connection : {'state': 'active', 'created_at': '2020-09-18 11:03:22.682793Z', 'connection_id': '97ef2879-7643-4e4f-9141-7d9edfa49de5', 'routing_state': 'none', 'their_did': 'MmdMDrUhYfHnF2yxH4Bfyb', 'invitation_mode': 'once', 'their_label': 'Alice', 'my_did': '5j6W5xo2aPPzXKKinSQc5E', 'initiator': 'self', 'updated_at': '2020-09-18 11:03:47.287292Z', 'invitation_key': 'EJv5a7NDdMZJ196SdmTpZtEt7VnmiFUvkpEVfdc4wnLu', 'accept': 'manual'}
Active Connection ID :  97ef2879-7643-4e4f-9141-7d9edfa49de5


### Instantiate Listener

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In this instance the handler decodes the sent attachment and stores it. 

In [4]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    base64_bytes = data.encode('ascii')
    text = base64.b64decode(base64_bytes)
    file_content = text.decode('ascii')
    filepath = f"received_files/{filename}"
    f = open(filepath,"w+")
    f.write(file_content)
    f.close()
    
#     data = payload['content'] 
#     asyncio.get_event_loop().create_task(om_controller.messaging.send_message(connection_id, " Received!"))
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([attach_listener], defaults=True)


In [31]:
def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    
    text = base64.b64decode(data)
    
    text.decode("utf-8")
    
    fh = open("image_received.png", "wb")
    fh.write(base64.b64decode(text))
    fh.close()
#     filepath = f"received_files/{filename}"
#     f = open(filepath,"w+")
#     f.write(file_content)
#     f.close()
    
#     data = payload['content'] 
#     asyncio.get_event_loop().create_task(om_controller.messaging.send_message(connection_id, " Received!"))
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([attach_listener], defaults=True)



Handle Attachment {'connection_id': '97ef2879-7643-4e4f-9141-7d9edfa49de5', 'message_id': '945e26e0-357d-494a-aedb-5113cbdfefb3', 'attachment_message': 'Here is the data you wanted', 'content': '...', 'filename': 'image.txt', 'state': 'received'} 97ef2879-7643-4e4f-9141-7d9edfa49de5
Attachment Received


In [8]:
response = await om_controller.protocol.send_attachment(connection_id,data)

print(response)


NameError: name 'data' is not defined


# End of Tutorial

Be sure to terminate the controller so you can run another tutorial.


In [25]:
response = await om_controller.terminate()
print(response)


None
